In [1]:
#Pytorch modules
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split, TensorDataset
from torchvision.datasets import CIFAR10
from torchvision import datasets, transforms, models
#scipy
from scipy.stats import mode
#sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
#Numpy
import numpy as np
#Pandas
import pandas as pd
#Lightning & logging
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
#Data observation
import os
import sys
import pickle
import requests
from pathlib import Path
#Plotting
import matplotlib.pyplot as plt
import seaborn as sns
#Logging
from clearml import Task

/Users/damirnurtdinov/miniconda3/envs/py39/lib/python3.9/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/damirnurtdinov/miniconda3/envs/py39/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <FB2FD416-6C4D-3621-B677-61F07C02A3C5> /Users/damirnurtdinov/miniconda3/envs/py39/lib/python3.9/site-packages/torchvision/image.so
  Reason: tried: '/Users/damirnurtdinov/miniconda3/envs/py39/lib/python3.9/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/damirnurtdinov/miniconda3/envs/py39/lib/python3.9/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/damirnurtdinov/miniconda3/envs/py39/lib/python3.9/lib-dynload/../../libjpeg.9.dylib' (no such file), '/Users/damirnurtdinov/miniconda3/envs/py39/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ign

In [2]:
# Function for setting the seed to implement parallel tests
SEEDS = [42, 0, 17, 9, 3, 16, 2]
SEED = 42 # random seed by default
pl.seed_everything(SEED)

# Determine the device (GPU if available, otherwise CPU)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device('cpu') # I have M1 chip

# Prioritizes speed but may reduce precision
torch.set_float32_matmul_precision('high')

# # Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

torch.manual_seed(SEED)
np.random.seed(SEED)

Seed set to 42


In [3]:
DATASET = 'CIFAR10N' # dataset with the real-world noise
# Can be 'clean_label', 'worse_label', 'aggre_label', 'random_label1', 'random_label2', 'random_label3'
NOISE_TYPE = 'worse_label'

NS = {
    'train': 45000,
    'val': 5000,
    'test': 10000
} # for MNIST

SIZE = 32 #image size
NUM_CLASSES = 10
CLASS_NAMES = ['plane', 'car', 'bird', 'cat',
               'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

#For the MNIST dataset
MEAN = np.array([0.491,0.482,0.447])
STD  = np.array([0.247,0.243,0.261])

#Model parameters
LOSS_FUN = 'N' # 'CE','CELoss'(custom), 'N', 'B', etc.
ARCHITECTURE = 'CNN' # 'CNN, 'ResNet50', 'ViT', etc.

#Collect the parameters (hyperparams and others)
hparams = {
    "seed": SEED,
    "lr": 0.001,
    'weight_decay': 0.0,
    "dropout": 0.0,
    "bs": 128,
    "num_workers": 0, #set 2 in Colab, or 0 in InnoDataHub
    "num_epochs": 20,
    "criterion": LOSS_FUN,
    "architecture": ARCHITECTURE,
    "num_samples": NS,
    "im_size": SIZE,
    "mean": np.array([0.4914, 0.4822, 0.4465]),
    "std": np.array([0.2470, 0.2435, 0.2616]),
    'randResCrop': {'size': (SIZE, SIZE), 'scale': (0.8, 1.0), 'ratio': (0.9, 1.1)},
    "n_classes": NUM_CLASSES,
    "noise_path": './data/CIFAR-10_human.pt',
    "noise_type": NOISE_TYPE  # Can be 'clean_label', 'worse_label', 'aggre_label', etc.
}

#Visualization
vis_params = {
    'fig_size': 5,
    'num_samples': 5,
    'num_bins': 50,
}

In [4]:
def download_file(url, save_path):
    """Download a file from a URL and save it to the specified path."""
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)  # Ensure directory exists
        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"File downloaded and saved to {save_path}")
    else:
        raise Exception(f"Failed to download file from {url}. Status code: {response.status_code}")
    
class CIFAR10(datasets.CIFAR10):
    """CIFAR10 dataset with noisy labels."""
    def __init__(self, root, train=True, transform=None, target_transform=None,
                 download=False, noise_type=None, noise_path=None, is_human=True):
        super().__init__(root, train=train, transform=transform,
                         target_transform=target_transform, download=download)
        self.noise_type = noise_type
        self.noise_path = noise_path
        self.is_human = is_human

        if self.train and self.noise_type is not None:
            self.load_noisy_labels()

    def load_noisy_labels(self):
        noise_file = torch.load(self.noise_path)
        if isinstance(noise_file, dict):
            if "clean_label" in noise_file.keys():
                clean_label = torch.tensor(noise_file['clean_label'])
                assert torch.sum(torch.tensor(self.targets) - clean_label) == 0
                print(f'Loaded {self.noise_type} from {self.noise_path}.')
                print(f'The overall noise rate is {1 - np.mean(clean_label.numpy() == noise_file[self.noise_type])}')
            self.noisy_labels = noise_file[self.noise_type].reshape(-1)
        else:
            raise Exception('Input Error')

    def __getitem__(self, index):
        img, target = super().__getitem__(index)
        if self.train and self.noise_type is not None:
            target = self.noisy_labels[index]
        return img, target, index
    
class CIFAR10DataModule(pl.LightningDataModule):
    def __init__(self, params):
        super().__init__()
        self.seed = params['seed']
        self.batch_size = params['bs']
        self.num_workers = params['num_workers']
        self.mean = params['mean']
        self.std = params['std']
        self.ns = params['num_samples']
        self.rand_res_crop = params['randResCrop']
        self.noise_path = params.get('noise_path', './data/CIFAR-10_human.pt')
        self.noise_type = params.get('noise_type', 'worse_label')  # Default to 'worse_label'

        # Ensure the data directory exists
        os.makedirs(os.path.dirname(self.noise_path), exist_ok=True)

        # Download the CIFAR-10_human.pt file if it doesn't exist
        if not os.path.exists(self.noise_path):
            print(f"Downloading CIFAR-10_human.pt from GitHub...")
            download_file(
                url="https://github.com/UCSC-REAL/cifar-10-100n/raw/main/data/CIFAR-10_human.pt",
                save_path=self.noise_path
            )

        self.transform = transforms.Compose([
            transforms.RandomResizedCrop(size=self.rand_res_crop['size'],
                                         scale=self.rand_res_crop['scale'],
                                         ratio=self.rand_res_crop['ratio']),
            transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std)
        ])

    def prepare_data(self):
        # Download CIFAR-10 dataset
        datasets.CIFAR10(root='./data', train=True, download=True)
        datasets.CIFAR10(root='./data', train=False, download=True)

    def setup(self, stage=None):
        # Load noisy labels
        noise_file = torch.load(self.noise_path)
        clean_label = noise_file['clean_label']
        noisy_label = noise_file[self.noise_type]

        # Split dataset into train and validation sets
        cifar10_full = CIFAR10(root='./data', train=True, transform=self.transform,
                               noise_type=self.noise_type, noise_path=self.noise_path, is_human=True)
        pl.seed_everything(self.seed)
        self.cifar10_train, self.cifar10_val = random_split(cifar10_full,
                                                            [self.ns['train'],
                                                             self.ns['val']])
        self.cifar10_test = CIFAR10(root='./data', train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.cifar10_train, batch_size=self.batch_size,
                          num_workers=self.num_workers, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.cifar10_val, batch_size=self.batch_size,
                          num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.cifar10_test, batch_size=self.batch_size,
                          shuffle=False)

In [5]:
def call_bn(bn, x):
    return bn(x)

class CNN(nn.Module):
    def __init__(self, input_channel=3, n_outputs=10, dropout_rate=0.25, top_bn=False):
        self.dropout_rate = dropout_rate
        self.top_bn = top_bn
        super(CNN, self).__init__()
        self.c1=nn.Conv2d(input_channel,128,kernel_size=3,stride=1, padding=1)
        self.c2=nn.Conv2d(128,128,kernel_size=3,stride=1, padding=1)
        self.c3=nn.Conv2d(128,128,kernel_size=3,stride=1, padding=1)
        self.c4=nn.Conv2d(128,256,kernel_size=3,stride=1, padding=1)
        self.c5=nn.Conv2d(256,256,kernel_size=3,stride=1, padding=1)
        self.c6=nn.Conv2d(256,256,kernel_size=3,stride=1, padding=1)
        self.c7=nn.Conv2d(256,512,kernel_size=3,stride=1, padding=0)
        self.c8=nn.Conv2d(512,256,kernel_size=3,stride=1, padding=0)
        self.c9=nn.Conv2d(256,128,kernel_size=3,stride=1, padding=0)
        self.l_c1=nn.Linear(128,n_outputs)
        self.bn1=nn.BatchNorm2d(128)
        self.bn2=nn.BatchNorm2d(128)
        self.bn3=nn.BatchNorm2d(128)
        self.bn4=nn.BatchNorm2d(256)
        self.bn5=nn.BatchNorm2d(256)
        self.bn6=nn.BatchNorm2d(256)
        self.bn7=nn.BatchNorm2d(512)
        self.bn8=nn.BatchNorm2d(256)
        self.bn9=nn.BatchNorm2d(128)

    def forward(self, x,):
        h=x
        h=self.c1(h)
        h=F.leaky_relu(call_bn(self.bn1, h), negative_slope=0.01)
        h=self.c2(h)
        h=F.leaky_relu(call_bn(self.bn2, h), negative_slope=0.01)
        h=self.c3(h)
        h=F.leaky_relu(call_bn(self.bn3, h), negative_slope=0.01)
        h=F.max_pool2d(h, kernel_size=2, stride=2)
        h=F.dropout2d(h, p=self.dropout_rate)

        h=self.c4(h)
        h=F.leaky_relu(call_bn(self.bn4, h), negative_slope=0.01)
        h=self.c5(h)
        h=F.leaky_relu(call_bn(self.bn5, h), negative_slope=0.01)
        h=self.c6(h)
        h=F.leaky_relu(call_bn(self.bn6, h), negative_slope=0.01)
        h=F.max_pool2d(h, kernel_size=2, stride=2)
        h=F.dropout2d(h, p=self.dropout_rate)

        h=self.c7(h)
        h=F.leaky_relu(call_bn(self.bn7, h), negative_slope=0.01)
        h=self.c8(h)
        h=F.leaky_relu(call_bn(self.bn8, h), negative_slope=0.01)
        h=self.c9(h)
        h=F.leaky_relu(call_bn(self.bn9, h), negative_slope=0.01)
        h=F.avg_pool2d(h, kernel_size=h.data.shape[2])

        h = h.view(h.size(0), h.size(1))
        logit=self.l_c1(h)
        if self.top_bn:
            logit=call_bn(self.bn_c1, logit)
        return logit
    
class ResNet50(nn.Module):
    def __init__(self, n_outputs, freeze=False):
        """
        Args:
            n_outputs (int): Number of output classes.
            freeze (bool): If True, freeze all layers except the head.
        """
        super(ResNet50, self).__init__()
        self.n_outputs = n_outputs
        self.freeze = freeze

        # Load the pre-trained ResNet50 model
        self.resnet50 = models.resnet50(pretrained=True)

        # Modify the final layer to match the number of outputs
        self.resnet50.fc = nn.Linear(self.resnet50.fc.in_features, n_outputs)

        # Freeze all layers except the head if freeze=True
        if self.freeze:
            self._freeze_layers()

    def _freeze_layers(self):
        """
        Freeze all layers except the head.
        """
        # Freeze all parameters in the model
        for param in self.resnet50.parameters():
            param.requires_grad = False

        # Unfreeze the final classification layer (head)
        for param in self.resnet50.fc.parameters():
            param.requires_grad = True

    def forward(self, x):
        return self.resnet50(x)

def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

class ViT(nn.Module):
    def __init__(self, n_outputs, freeze=False):
        """
        Args:
            n_outputs (int): Number of output classes.
            freeze (bool): If True, freeze all layers except the head.
        """
        super(ViT, self).__init__()
        self.n_outputs = n_outputs
        self.freeze = freeze

        # Load the pre-trained ViT model
        self.vit = models.vit_b_16(pretrained=True)

        # Modify the final layer to match the number of outputs
        self.vit.heads.head = nn.Linear(self.vit.heads.head.in_features, n_outputs)

        # Freeze all layers except the head if freeze=True
        if self.freeze:
            self._freeze_layers()

    def _freeze_layers(self):
        """
        Freeze all layers except the head.
        """
        # Freeze all parameters in the model
        for param in self.vit.parameters():
            param.requires_grad = False

        # Unfreeze the head (final classification layer)
        for param in self.vit.heads.head.parameters():
            param.requires_grad = True

    def forward(self, x):
        return self.vit(x)


In [6]:
def metrics(dataloader,model,hparams=hparams,loss_fn_red=None):
    # Collect images, predictions, and losses
    # images = []
    preds  = []
    labels = []
    losses = []
    correct= 0
    total  = 0
    for batch in dataloader:
        x, y, _ = batch
        with torch.no_grad():
            logits = model(x)
            # loss = loss_fn_red(h,y)
            pred = torch.argmax(logits[:,:hparams['n_classes']], dim=1)
        correct += (pred == y).sum().item()  # Number of correct predictions
        total += y.size(0)  # Total number of samples

        # images.extend(x.cpu())
        preds.extend(pred.cpu().numpy())
        labels.extend(y.cpu().numpy())
        # losses.extend(loss.cpu().numpy())
    acc = correct / total
    return preds, labels, acc

In [7]:
data_module = CIFAR10DataModule(hparams)
data_module.prepare_data()
data_module.setup()
test_dataloader = data_module.test_dataloader()

Files already downloaded and verified
Files already downloaded and verified


/var/folders/6j/s19nhfjj4pv14vny7nrq8lf00000gn/T/ipykernel_6271/2517412809.py:83: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  noise_file = torch.load(self.noise_path)
/var

Loaded worse_label from ./data/CIFAR-10_human.pt.
The overall noise rate is 0.40208


In [8]:
class train_model(pl.LightningModule):
    def __init__(self, model=None, loss=None, hparams=hparams):
        super().__init__()
        self.save_hyperparameters(hparams)
        self.model = model
        self.loss_fn = loss
        self.nc = hparams['n_classes']
        self.lr = hparams['lr']
        self.wd = hparams['weight_decay']

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y, _ = batch  # Unpack batch (ignore indices for now)
        logits = self(x)
        loss = self.loss_fn(logits, y)

        # Log training loss and accuracy
        # preds = torch.argmax(logits[:, :self.nc], dim=1)
        # acc = (preds == y).float().mean()
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        # self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y, _ = batch  # Unpack batch (ignore indices for now)
        logits = self(x)
        loss = self.loss_fn(logits, y)

        # Log validation loss and accuracy
        # preds = torch.argmax(logits[:, :self.nc], dim=1)
        # acc = (preds == y).float().mean()
        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        # self.log('val_acc', acc, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y, _ = batch  # Unpack batch (ignore indices for now)
        logits = self(x)
        loss = self.loss_fn(logits, y)

        # Log test loss and accuracy
        preds = torch.argmax(logits[:, :self.nc], dim=1)
        acc = (preds == y).float().mean()
        self.log('test_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('test_acc', acc, on_step=True, on_epoch=True, prog_bar=True)
        return {'loss': loss, 'preds': preds, 'y': y}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr, weight_decay=self.wd)

        # Optionally, add a learning rate scheduler
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=1.0)
        return [optimizer], [scheduler]

In [9]:
class BLoss(nn.Module):
    def __init__(self, params=hparams):
        super(BLoss, self).__init__()
        self.smoothing = params.get('label_smoothing', 0.0)
        self.num_classes = params.get('n_classes', 10)
        self.inv_smoothing = 1.0 - self.smoothing  # Probability for the correct class
        self.eps = 1e-10  # Small epsilon for numerical stability

    def forward(self, x, y):
        """
        x: Model output (logits + certainty)
            - x[:, :self.num_classes]: Logits for class probabilities
            - x[:, self.num_classes:]: Certainty values
        y: Ground truth labels (class indices)
        """
        # Ensure y is a tensor of class indices
        if y.dtype != torch.long:
            y = y.long()

        # Extract certainty and probabilities from the model output
        certainty = torch.sigmoid(x[:, self.num_classes:])  # Certainty values (batch_size, 1)
        logits = x[:, :self.num_classes]  # Logits for class probabilities (batch_size, num_classes)
        prob = F.softmax(logits, dim=1)  # Softmax probabilities (batch_size, num_classes)

        # Apply label smoothing to the one-hot encoded labels
        with torch.no_grad():
            yoh = torch.zeros_like(logits)
            yoh.fill_(self.smoothing / (self.num_classes - 1))
            yoh.scatter_(1, y.unsqueeze(1), self.inv_smoothing)

        # Compute cosine similarity between predictions and labels
        cos = nn.CosineSimilarity(dim=1)
        cosyh = cos(yoh, prob)  # Cosine similarity (batch_size,)

        # Compute the terms of the loss
        delta = yoh * prob  # Element-wise product of one-hot labels and probabilities
        entropy_term = delta * torch.log(delta + self.eps)  # Entropy term (avoid log(0))

        # Loss terms
        loss0 = -cosyh * torch.log(certainty.squeeze() / self.num_classes + self.eps)  # First term
        loss1 = -(self.num_classes - 1) * (1 - cosyh) * torch.log((1 - certainty.squeeze()) / self.num_classes + self.eps)  # Second term

        # Combine the terms and compute the mean over the batch
        loss = (entropy_term.sum(dim=1) + loss0 + loss1).mean()

        return loss
    
class NLoss(nn.Module):
    def __init__(self, params=hparams):
        super(NLoss, self).__init__()
        self.smoothing =   params.get('label_smoothing', 0.0)
        self.num_classes = params.get('n_classes', 10)
        self.inv_smoothing = 1.0 - self.smoothing  # Probability for the correct class

    def forward(self, x, y):
        """
        x: Model output (logits + log variance)
            - x[:, :self.num_classes]: Logits for class probabilities (h)
            - x[:, self.num_classes:]: Logarithmic variance (s)
        y: Labels
        """
        # Split the model output into predictions (h) and log variance (s)
        logits = x[:, :self.num_classes]  # Predictions (h)
        log_var = x[:, self.num_classes:]  # Logarithmic variance (s)

        # Apply label smoothing to the one-hot encoded labels
        with torch.no_grad():
            yoh = torch.zeros_like(logits)
            yoh.fill_(self.smoothing / (self.num_classes - 1))
            yoh.scatter_(1, y.data.unsqueeze(1), self.inv_smoothing)

        # Compute the squared differences between predictions and smoothed labels
        # logits = torch.softmax(logits, dim=1)  # Convert logits to probabilities
        squared_diff = torch.pow(yoh - logits, 2)  # (y_k - h_k)^2

        # Compute the exponential of the negative log variance (e^{-s})
        # log_var = torch.clamp(log_var, min=-10, max=10)  # Clamp log_var to a reasonable range
        exp_neg_log_var = torch.exp(-log_var)

        # Compute the first term of the loss: e^{-s} * sum((y_k - h_k)^2)
        term1 = exp_neg_log_var * squared_diff.sum(dim=1)

        # Compute the second term of the loss: N * s
        term2 = self.num_classes * log_var

        # Combine the terms and compute the mean over the batch
        loss = (term1 + term2).mean()

        return loss

In [10]:
def get_arch_and_loss(hparams):
    """
    Returns the architecture and loss function based on the provided hparams.

    Args:
        hparams (dict): Hyperparameters dictionary, including 'ARCHITECTURE' and 'criterion'.

    Returns:
        arch: The model architecture.
        loss: The loss function.
    """
    # Determine the number of outputs based on the loss function
    if hparams['criterion'] in ['B', 'N']:
        n_outputs = hparams['n_classes'] + 1  # Add 1 output neuron for BLoss or NLoss
    else:
        n_outputs = hparams['n_classes']  # Default number of outputs
    # print(n_outputs)
    # Define the architectures
    architectures = {
        'CNN': CNN(n_outputs=n_outputs),
        'ResNet50': ResNet50(n_outputs=n_outputs, freeze=hparams.get('freeze', False)),
        'ViT': ViT(n_outputs=n_outputs, freeze=hparams.get('freeze', False)),
    }

    # Define the loss functions
    losses = {
        'CE': nn.CrossEntropyLoss(),
        'B': BLoss(),
        'N': NLoss(),
    }

    # Get the architecture and loss based on hparams
    arch = architectures.get(hparams['architecture'])
    loss = losses.get(hparams['criterion'])

    if arch is None:
        raise ValueError(f"Architecture '{hparams['ARCHITECTURE']}' is not supported.")
    if loss is None:
        raise ValueError(f"Loss function '{hparams['criterion']}' is not supported.")

    return arch, loss


In [11]:
import glob
import os

architectures = {
        'CNN': CNN(n_outputs=10),
        'ResNet50': ResNet50(n_outputs=10, freeze=hparams.get('freeze', False)),
        'ViT': ViT(n_outputs=10, freeze=hparams.get('freeze', False)),
    }

losses = {
    'CE': nn.CrossEntropyLoss(),
    'B': BLoss(),
    'N': NLoss(),
}

# Function to load a model from a checkpoint
def load_model(architecture, loss, seed, hparams):
    # Use glob to find all matching checkpoint files
    pattern = f"checkpoints/model-{architecture}-{loss}-seed-{seed}-epoch=*.ckpt"
    matching_files = glob.glob(pattern)
    
    if not matching_files:
        raise FileNotFoundError(f"No checkpoint found for {architecture}, {loss}, seed {seed}")
    
    # Load the first matching file (or choose the latest epoch if needed)
    model_path = matching_files[0]
    
    arch, loss_fn = get_arch_and_loss(hparams)
    
    # Instantiate the model and load the checkpoint
    # model = architectures[architecture]
    # loss_fn = losses[loss]
    best_model = train_model.load_from_checkpoint(model_path, model=arch, loss=loss_fn)
    best_model.eval()
    return best_model

/Users/damirnurtdinov/miniconda3/envs/py39/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/damirnurtdinov/miniconda3/envs/py39/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/Users/damirnurtdinov/miniconda3/envs/py39/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGEN

In [12]:
# Function to get predictions from a model
# def get_predictions(model, dataloader):
#     trainer = pl.Trainer(accelerator='mps', devices=-1)
#     test_results = trainer.test(model, dataloaders=dataloader)
    
#     # Extract predictions from the test results
#     preds = torch.cat([x['preds'] for x in test_results])
#     return preds.cpu().numpy()

from tqdm import tqdm

def get_predictions(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    device = next(model.parameters()).device  # Get the device of the model
    all_preds = []

    with torch.no_grad():  # Disable gradient computation
        # Wrap the dataloader with tqdm for a progress bar
        for batch in tqdm(dataloader, desc="Processing batches", leave=False):
            inputs = batch[0].to(device)  # Move inputs to the correct device
            outputs = model(inputs)  # Forward pass
            _, preds = torch.max(outputs, 1)  # Get predicted class indices
            all_preds.append(preds.cpu())  # Move predictions to CPU and store

    # Concatenate all predictions into a single tensor
    return torch.cat(all_preds).numpy()

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
from scipy.stats import mode
architectures_list = [ 'CNN'] # TODO: check for ViT
losses_list = ['N','B','CE']
seeds = [42, 0, 17, 9, 3, 16, 2]


# Function to load and test a model using PyTorch Lightning
def load_and_test_model(arch, loss, seed, hparams, test_dataloader):
    # Use glob to find all matching checkpoint files
    pattern = f"checkpoints/model-{arch}-{loss}-seed-{seed}-epoch=*.ckpt"
    matching_files = glob.glob(pattern)
    
    if not matching_files:
        raise FileNotFoundError(f"No checkpoint found for {arch}, {loss}, seed {seed}")
    
    # Load the first matching file (or choose the latest epoch if needed)
    model_path = matching_files[0]
    
    # Get the architecture and loss function based on hparams
    arch, loss_fn = get_arch_and_loss(hparams)
    
    # Load the model from the checkpoint
    best_model = train_model.load_from_checkpoint(model_path, model=arch, loss=loss_fn)
    best_model = best_model.to(device)
    
    # Test the model using PyTorch Lightning's Trainer
    trainer = pl.Trainer(accelerator='mps', devices=-1)
    test_results = trainer.test(best_model, dataloaders=test_dataloader)
    
    # Extract predictions from the test results
    preds = torch.cat([x['preds'] for x in test_results])
    return preds.cpu().numpy()

# Dictionary to store individual accuracies for each architecture, loss, and seed
individual_accuracies = {arch: {loss: [] for loss in losses_list} for arch in architectures_list}

# Dictionary to store predictions for each architecture
architecture_predictions = {arch: [] for arch in architectures_list}

# Loop through all models and collect predictions
for arch in tqdm(architectures_list, desc="Architectures"):
    for loss in tqdm(losses_list, desc="Losses", leave=False):
        for seed in tqdm(seeds, desc="Seeds", leave=False):
            try:
                # Update hparams for the current architecture and loss
                hparams['architecture'] = arch
                hparams['criterion'] = loss
                
                # Load and test the model
                predictions = load_and_test_model(arch, loss, seed, hparams, test_dataloader)
                architecture_predictions[arch].append(predictions)
                
                # Calculate accuracy for the current model
                true_labels = np.array(data_module.cifar10_test.targets)
                accuracy = accuracy_score(true_labels, predictions)
                individual_accuracies[arch][loss].append(accuracy)
                print(f'{arch} {loss} seed {seed} Accuracy: {accuracy:.4f}')
            except FileNotFoundError as e:
                print(e)
                continue

# Compute mean accuracy and standard deviation for each architecture and loss
for arch in architectures_list:
    for loss in losses_list:
        accuracies = individual_accuracies[arch][loss]
        if accuracies:  # Check if there are any accuracies for this combination
            mean_accuracy = np.mean(accuracies)
            std_accuracy = np.std(accuracies)
            print(f'{arch} {loss} Mean Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}')

# Compute ensemble accuracy for each architecture
for arch in architectures_list:
    if architecture_predictions[arch]:  # Check if there are any predictions for this architecture
        all_predictions = np.stack(architecture_predictions[arch])  # Shape: (num_models, num_samples)
        
        # Ensemble predictions (e.g., by majority voting)
        ensemble_predictions, _ = mode(all_predictions, axis=0)  # Majority voting
        ensemble_predictions = ensemble_predictions.flatten()  # Flatten to 1D array

        # Get true labels from the CIFAR-10 dataset
        test_labels = np.array(data_module.cifar10_test.targets)

        # Calculate ensemble accuracy
        accuracy = accuracy_score(test_labels, ensemble_predictions)
        print(f'{arch} Ensemble Accuracy: {accuracy:.4f}')

        # Compute confusion matrix
        cm = confusion_matrix(test_labels, ensemble_predictions)
        print(f'{arch} Confusion Matrix:\n{cm}')

Architectures:   0%|          | 0/1 [00:00<?, ?it/s]
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/damirnurtdinov/miniconda3/envs/py39/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/Users/damirnurtdinov/miniconda3/envs/py39/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_w

Testing DataLoader 0:   4%|▍         | 3/79 [00:48<20:31,  0.06it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...
